<a href="https://colab.research.google.com/github/redars/computer_alg/blob/main/Task_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from IPython.display import display, Math
import matplotlib
import matplotlib.pyplot as plt

In [18]:
def add(x, y):
  return ['+', x, y]

def mul(x, y):
  return ['*', x, y]

def div(x, y):
  return ["/", x, y]

def sub(x, y):
  return ["-", x, y]

def pow(x, y):
  return ["^", x, y]

def neg(x):
  return ["--", x]

def eq(x, y):
  return ["=", x, y]


In [20]:
def fsqrt(x):
  return ['sqrt', x]

def fsin(x):
  return ['sin', x]

def fcos(x):
  return ['cos', x]

def fln(x):
  return ['ln', x]

In [21]:
def check(x):
  if isinstance(x, Expression):
    return x
  return Expression(x)

class Expression:
  def __init__(self, f):
    self.f = f
  def __add__(self, x):
    x = check(x)
    return Expression(add(self.f, x.f))
  def __sub__(self, x):
    x = check(x)
    return Expression(sub(self.f, x.f))
  def __mul__(self, x):
    x = check(x)
    return Expression(mul(self.f, x.f))
  def __truediv__(self, x):
    x = check(x)
    return Expression(div(self.f, x.f))
  def __pow__(self, x):
    x = check(x)
    return Expression(pow(self.f, x.f))
  def __radd__(self, x):
    x = check(x)
    return Expression(add(x.f, self.f))
  def __rsub__(self, x):
    x = check(x)
    return Expression(sub(x.f, self.f))
  def __rmul__(self, x):
    x = check(x)
    return Expression(mul(x.f, self.f))
  def __rtruediv__(self, x):
    x = check(x)
    return Expression(div(x.f, self.f))
  def __rpow__(self, x):
    x = check(x)
    return Expression(pow(x.f, self.f))
  def __neg__(self):
    return Expression(neg(self.f))
  def subs(self, var, val):
    return Expression(substitute(self.f, var, val))
  def evalf(self):
    return evaluate(self.f)
  def __str__(self):
    return expr2latex(self.f)
  def show(self):
    display(Math(expr2latex(self.f)))
  def diff(self, var):
    var = check(var).f
    return Expression(differentiate(self.f, var))
  def simp(self):
    return Expression(simplify(self.f))
  def expn(self):
    return Expression(expand(self.f))
  def intg(self, var):
    var = check(var).f
    return Expression(integrate(self.f, var))
  def __eq__(self, x):
    x = check(x)
    return Expression(eq(self.f, x.f))
  def solve(self, var):
    var = check(var).f
    return Expression(solver(self.f, var))
  def dsolve(self, y, x, init = None):
    x = check(x).f
    y = check(y).f
    return Expression(dsolver(self.f, y, x, init))

In [22]:
def sqrt(x):
  x = check(x)
  t = x.f
  r = fsqrt(t)
  return Expression(r)

def sin(x):
  return Expression(fsin(check(x).f))

def cos(x):
  return Expression(fcos(check(x).f))

def ln(x):
  return Expression(fln(check(x).f))


In [23]:
def substitute(f, var, val):
  var = check(var).f
  val = check(val).f
  if f == var:
    return val
  elif isinstance(f, list):
    y = [f[0]]
    for i in range(1, len(f)):
      t = substitute(f[i], var, val)
      y.append(t)
    return y
  else:
    return f


In [24]:
import math

def evaluate(f):
  if isinstance(f, (int, float)):
    return f
  elif isinstance(f, list):
    if f[0] == "+": 
      return evaluate(f[1]) + evaluate(f[2])
    elif f[0] == "*":
      return evaluate(f[1]) * evaluate(f[2])
    elif f[0] == "-": 
      return evaluate(f[1]) - evaluate(f[2])
    elif f[0] == "/":
      return evaluate(f[1]) / evaluate(f[2])
    elif f[0] == "sqrt":
      return math.sqrt(evaluate(f[1]))
    elif f[0] == "sin":
      return math.sin(evaluate(f[1]))
    elif f[0] == "cos":
      return math.cos(evaluate(f[1]))
  return None
  

In [25]:
def plot(expr, var, a, b, n):
  dx = (b - a) / n
  X, Y = [], []
  for i in range(n + 1):
    x = a + dx * i
    y = expr.subs(var, x).evalf()
    X.append(x)
    Y.append(y)

  fig, ax = plt.subplots(figsize=(12, 9))
  ax.plot(X, Y, color="red", lw=5)
  plt.show()


In [26]:
def symbols(vars):
  return map(Expression, vars.split())

x, y = symbols("x y")  

In [49]:
def expr2latex(x):
  if isinstance(x, (int, float)):
    return str(x)
  if isinstance(x, str):
    return x
  if isinstance(x, list):
    if x[0] == "+":
      return expr2latex(x[1]) + "+" + expr2latex(x[2])
    if x[0] == "-":
      return expr2latex(x[1]) + "-" + expr2latex(x[2])
    if x[0] == "*":
      left = expr2latex(x[1])
      if isinstance(x[1], list) and x[1][0] == "+":
        left = "\\left(" + left + "\\right)"
      right = expr2latex(x[2])
      if isinstance(x[2], list) and x[2][0] == "+":
        right = "\\left(" + right + "\\right)"
      return left + "\\cdot " + right
    if x[0] == "/":
      return "\\dfrac{" + expr2latex(x[1]) + "}{" + expr2latex(x[2]) + "}"
    if x[0] == "^":
      left = expr2latex(x[1])
      if isinstance(x[1], list):
        left = "\\left(" + left + "\\right)"
      right = expr2latex(x[2])
      return "{" + left + "}^{" + right + "}"
    if x[0] == "--":
      a = expr2latex(x[1])
      if isinstance(x[1], list):
        a = "\\left(" + a + "\\right)"
      return "-{" + a + "}"
    if x[0] == "sqrt":
      a = expr2latex(x[1])
      return "\\sqrt{" + a + "}"
    if x[0] == "sin":
      a = expr2latex(x[1])
      if isinstance(x[1], list):
        a = "\\left(" + a + "\\right)"
      return "\\sin{" + a + "}"
    if x[0] == "cos":
      a = expr2latex(x[1])
      if isinstance(x[1], list):
        a = "\\left(" + a + "\\right)"
      return "\\cos{" + a + "}"
    if x[0] == "ln":
      a = expr2latex(x[1])
      if isinstance(x[1], list):
        a = "\\left(" + a + "\\right)"
      return "\\ln{" + a + "}"
    if x[0] == "int":
      a = expr2latex(x[1])
      if isinstance(x[1], list) and x[1][0] in "+-":
        a = "\\left(" + a + "\\right)"
      return "\\int{" + a + "}d" + x[2]
    if x[0] == "=":
      return expr2latex(x[1]) + "=" + expr2latex(x[2])
    if x[0] == "D": 
      a = expr2latex(x[1])
      if isinstance(x[1], list):
        a = "\\left(" + a + "\\right)"
      b = expr2latex(x[2])
      if isinstance(x[2], list):
        b = "\\left(" + b + "\\right)"
      return "\\dfrac{d{" + a + "}}{d{" + b + "}}"


In [28]:
def differentiate(f, x):
  if isinstance(f, int):
    return 0
  if f == x:
    return 1
  if isinstance(f, str):
    return 0
  if isinstance(f, list):
    if f[0] == "+":
      u, v = f[1], f[2]
      u1, v1 = differentiate(u, x), differentiate(v, x)
      return add(u1, v1)
    if f[0] == "-":
      u, v = f[1], f[2]
      u1, v1 = differentiate(u, x), differentiate(v, x)
      return sub(u1, v1)      
    if f[0] == "*":
      u, v = f[1], f[2]
      u1, v1 = differentiate(u, x), differentiate(v, x)
      return add(mul(u1, v), mul(u, v1))   
    if f[0] == "/":
      u, v = f[1], f[2]
      u1, v1 = differentiate(u, x), differentiate(v, x)
      return div(sub(mul(u1, v), mul(u, v1)), pow(v, 2))   
    if f[0] == "^":
      u, v = f[1], f[2]
      u1, v1 = differentiate(u, x), differentiate(v, x)
      p1, p2 = pow(u, v), pow(u, sub(v, 1))
      s1 = mul(p1, mul(v1, fln(u)))
      s2 = mul(v, mul(p2, u1))
      return add(s1, s2) 
    if f[0] == "--":
      u = f[1]
      u1 = differentiate(u, x)
      return neg(u1)
    if f[0] == "ln":
      u = f[1]
      u1 = differentiate(u, x)
      return div(u1, u)
    if f[0] == "sin":
      u = f[1]
      u1 = differentiate(u, x)
      return mul(fcos(u), u1)
    if f[0] == "cos":
      u = f[1]
      u1 = differentiate(u, x)
      return mul(neg(fsin(u)), u1)
    if f[0] == "sqrt":
      u = f[1]
      u1 = differentiate(u, x)
      return div(u1, mul(2, fsqrt(u)))
      

In [29]:
def simplify(f):
  if isinstance(f, list):
    r = [f[0]]
    for x in f[1:]:
      r.append(simplify(x))
    if r[0] == "+":
      if isinstance(r[1], int) and isinstance(r[2], int):
        return r[1] + r[2] 
      if r[1] == 0: 
        return r[2]
      if r[2] == 0:
        return r[1]
      if r[1] == r[2]:
        return mul(2, r[1])
      if isinstance(r[2], list) and r[2][0] == "--": # unary minus
        return simplify(sub(r[1], r[2][1]))
    if r[0] == "-":
      if isinstance(r[1], int) and isinstance(r[2], int):
        return r[1] - r[2]
      if r[1] == 0: 
        return neg(r[2])
      if r[2] == 0:
        return r[1]
      if r[1] == r[2]:
        return 0
      if isinstance(r[2], list) and r[2][0] == "--": # unary minus
        return simplify(add(r[1], r[2][1]))
    if r[0] == "*":
      if isinstance(r[1], int) and isinstance(r[2], int):
        return r[1] * r[2] 
      if r[1] == 0 or r[2] == 0: 
        return 0
      if r[1] == 1:
        return r[2]
      if r[2] == 1:
        return r[1]
      if r[1] == r[2]:
        return pow(r[1], 2)
    if r[0] == "/":
      if r[2] == 1:
        return r[1]
    if r[0] == "^":
      if isinstance(r[1], int) and isinstance(r[2], int) and r[2] > 0:
        return r[1] ** r[2] 
      if r[2] == 1: 
        return r[1]
    if r[0] == "sqrt":
      if isinstance(r[1], int):
        a = int(math.sqrt(r[1]))
        if a * a == r[1]:
          return a
        return r 
    return r
  return f 

In [30]:
def expand(f):
  if isinstance(f, list):
    r = [f[0]]
    for x in f[1:]:
      r.append(expand(x))
    if r[0] == "*":
      if isinstance(r[1], list) and r[1][0] == "+":
        a = expand(mul(r[1][1], r[2]))
        b = expand(mul(r[1][2], r[2]))
        return add(a, b)
      if isinstance(r[2], list) and r[2][0] == "+":
        a = expand(mul(r[1], r[2][1]))
        b = expand(mul(r[1], r[2][2]))
        return add(a, b)
    if r[0] == "^":
      if isinstance(r[1], list) and r[1][0] == "+" and r[2] == 2:
        a = pow(r[1][1], 2)
        b = mul(2, mul(r[1][1], r[1][2]))
        c = pow(r[1][2], 2)
        return add(a, add(b, c))
    return r
  return f


In [46]:
def integral(f, x):
  return ["int", f, x]

def contains(f, x):
  if f == x:
    return True
  if isinstance(f, (int, str)):
    return False
  if isinstance(f, list):
    for a in f:
      if contains(a, x):
        return True
  return False

def integrate(f, x):
  if not contains(f, x):
    return mul(f, x)
  if f == x:
    return mul(div(1, 2), pow(x, 2))
  if isinstance(f, list):
    if f[0] == "cos" and f[1] == x:
      return fsin(x)
    if f[0] == "sin" and f[1] == x:
      return neg(fcos(x))
    if f[0] == "sqrt" and f[1] == x:
      return mul(div(2, 3), mul(x, fsqrt(x)))
    if f[0] == "^" and f[1] == x and not contains(f[2], x):
      if f[2] == -1:
        return fln(x)
      else:
        return mul(div(1, add(f[2], 1)), pow(x, add(f[2], 1))) 
    if f[0] == "^" and f[2] == x and not contains(f[1], x):
      return mul(div(1, fln(f[1])), f)
    if f[0] in "+-":
      a = integrate(f[1], x)
      b = integrate(f[2], x)
      return [f[0], a, b]
    if f[0] == "*" and not contains(f[1], x):
      a = integrate(f[2], x)
      return mul(f[1], a)
    if f[0] == "*" and not contains(f[2], x):
      a = integrate(f[1], x)
      return mul(f[2], a)
    if f[0] == "/" and not contains(f[2], x):
      a = integrate(f[1], x)
      return mul(div(1, f[2]), a)
    if f[0] == "/" and f[1] == 1 and f[2] == 'x':
      return fln(x)
    if f[0] == "/" and not contains(f[1], x) and f[2][0] == '^' and f[2][1] == 'x':
      if f[2][2] != 1:
        p = f[2][2]
        p = p-1
        return div(-p,pow(x,p))
    if f[0] == "/" and f[1] == 1 and f[2] == 'y':
      return fln(y)
    if f[0] == "/" and not contains(f[1], y) and f[2][0] == '^' and f[2][1] == 'y':
      if f[2][2] != 1:
        p = f[2][2]
        p = p-1
        return div(-p,pow(y,p))
    if f[0] == "*" and f[1] == x:
      t = x + "t"
      g = substitute(f[2], pow(x, 2), t)
      #Expression(g).show()
      if not contains(g, x):
        G = integrate(g, t)
        if not contains(G, "int"): 
          F = substitute(G, t, pow(x, 2))
          return mul(div(1, 2), F)

  return integral(f, x)

In [33]:
z = x + 1 == 2
print(z.f)
z.show()

['=', ['+', 'x', 1], 2]


<IPython.core.display.Math object>

In [34]:
def terms(f): 
  if isinstance(f, list) and f[0] == "+":
    p1, n1 = terms(f[1])
    p2, n2 = terms(f[2])
    return p1 + p2, n1 + n2
  if isinstance(f, list) and f[0] == "-":
    p1, n1 = terms(f[1])
    p2, n2 = terms(f[2]) # (p1 - n1) - (p2 - n2) = p1 - n1  - p2 + n2
    return p1 + n2, n1 + p2
  return [f], [] 

def factors(f): 
  if isinstance(f, list) and f[0] == "*":
    n1, d1 = factors(f[1])
    n2, d2 = factors(f[2]) # (n1 / d1) * (n2 / d2) --> (n1 * n2) / (d1 * d2)
    return n1 + n2, d1 + d2
  if isinstance(f, list) and f[0] == "/":
    n1, d1 = factors(f[1])
    n2, d2 = factors(f[2]) # (n1 / d1) / (n2 / d2) --> (n1 * d2) / (d1 * n2)
    return n1 + d2, d1 + n2
  return [f], [] 

def Mul(L):
  res = L[0]
  for i in L[1:]:
    res = mul(res, i)
  return res

def solver(f, x):
  ls, rs = f[1], f[2]
  if contains(rs, x):
    ls, rs = rs, ls

  p, n = terms(ls)
  for i in p:
    if contains(i, x):
      ls = i
    else:
      rs = sub(rs, i)
  for i in n:
    if contains(i, x):
      ls = mul(-1, i)
    else:
      rs = add(rs, i)

  if ls == x:
    ls = simplify(ls)
    rs = simplify(rs)
    return eq(ls, rs) # solved!

  nl, dl = factors(ls)
  nr, dr = factors(rs)
  for i in nl:
    if contains(i, x):
      ls = i
      down = False
    else:
      dr.append(i)
  for i in dl:
    if contains(i, x):
      ls = i
      down = True
    else:
      nr.append(i)
  if down:
    nr, dr = dr, nr
  if dr == []:
    rs = Mul(nr)  
  else:
    rs = div(Mul(nr), Mul(dr))

  if ls == x:
    ls = simplify(ls)
    rs = simplify(rs)
    return eq(ls, rs) # solved!

  if isinstance(ls, list) and ls[0] in "+-":
    return solver(eq(ls, rs), x)  # recursive call

  ls = simplify(ls)
  rs = simplify(rs)
  return eq(ls, rs)

In [36]:
def derivative(y, x):
  return ["D", y, x]

def D(y, x):
  x = check(x).f
  y = check(y).f
  return Expression(derivative(y, x))

def dsolver(ode, y, x, init):
  d = derivative(y, x) # ["D", y, x]
  ode2 = solver(ode, d) # ["=", ["D", y, x], rhs]
  print("Разрешение производной")
  Expression(ode2).show()
  if ode2[1] != d:
    return ode2

  n, d = factors(ode2[2])

  nx, dx = [1], [1]
  ny, dy = [1], [1]

  for i in n:
    if contains(i, y):
      dy.append(i)
    else:
      nx.append(i)
  
  for i in d:
    if contains(i, x):
      dx.append(i)
    else:
      ny.append(i)

  yf = simplify(div(Mul(ny), Mul(dy)))
  xf = simplify(div(Mul(nx), Mul(dx)))
  print("Разделение переменных")
  (Expression(yf) * Expression("dy") == Expression(xf) * Expression("dx")).show()

  YF = simplify(integrate(yf, y))
  XF = simplify(integrate(xf, x))  

  if init == None:
    return eq(YF, add(XF, "C"))
  else:
    y0, x0 = init
    YF0 = simplify(substitute(YF, y, y0))
    XF0 = simplify(substitute(XF, x, x0))
    return eq(sub(YF, YF0), sub(XF, XF0))

In [37]:
z = 2 * D(y, x) == x + 2
z.show()
z.dsolve(y, x) 

<IPython.core.display.Math object>

Разрешение производной


<IPython.core.display.Math object>

Разделение переменных


<IPython.core.display.Math object>

In [54]:
z = 1 / x * D(y, x) == 1 / y ** 2
z.show()
q = z.dsolve(y, x)
print("Решение")
q.show()
#q.solve(y).show()

<IPython.core.display.Math object>

Разрешение производной


<IPython.core.display.Math object>

Разделение переменных


<IPython.core.display.Math object>

Решение


<IPython.core.display.Math object>

In [39]:
z = y == x / (cos(y ** 2) * D(y, x))
z.show()
z.dsolve(y, x).show()

<IPython.core.display.Math object>

Разрешение производной


<IPython.core.display.Math object>

Разделение переменных


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [40]:
z = x * D(y, x) == y
z.show()
z.dsolve(y, x).show()

<IPython.core.display.Math object>

Разрешение производной


<IPython.core.display.Math object>

Разделение переменных


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [41]:
z = y ** -1 * D(y, x) == x ** -1
z.show()
z.dsolve(y, x).show()

<IPython.core.display.Math object>

Разрешение производной


<IPython.core.display.Math object>

Разделение переменных


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [42]:
z = 1 / x * D(y, x) == 1 / y ** 2
z.show()
q = z.dsolve(y, x, (2, 1))
print("Решение")
q.show()

<IPython.core.display.Math object>

Разрешение производной


<IPython.core.display.Math object>

Разделение переменных


<IPython.core.display.Math object>

Решение


<IPython.core.display.Math object>

In [43]:
z = 1 / x * D(y, x) == 1 / y ** 2
z.show()
a, b = symbols("a b")
z.dsolve(y, x, (b, a)).show() # начальное условие: y(a) = b

<IPython.core.display.Math object>

Разрешение производной


<IPython.core.display.Math object>

Разделение переменных


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [58]:
z = 2 * D(y, x) == x + a 
z.show()
q = z.dsolve(y, x)
q.show()
q.solve(y).show() 

<IPython.core.display.Math object>

Разрешение производной


<IPython.core.display.Math object>

Разделение переменных


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [62]:
z = x*x* D(y, x) == y
z.show()
q = z.dsolve(y, x)
q.show()

<IPython.core.display.Math object>

Разрешение производной


<IPython.core.display.Math object>

Разделение переменных


<IPython.core.display.Math object>

TypeError: ignored

In [ ]:
z = 1/x**3
z.intg(x).show()
z = 1/x
z.intg(x).show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>